In [ ]:

# Plotly compatibility fix
def fix_plotly_data(data):
    """Convert range/arange objects to lists for Plotly compatibility"""
    if isinstance(data, range):
        return list(data)
    elif isinstance(data, np.ndarray):
        return data.tolist()
    elif hasattr(data, 'tolist'):
        return data.tolist()
    return data


In [ ]:
"""
Government Finance and Fiscal Policy Analysis - Kenya's Public Financial Management
==================================================================================

Comprehensive analysis of Kenya's government finances and fiscal policy including:
- Revenue and expenditure analysis
- Public debt sustainability assessment
- Fiscal balance and deficit financing
- Domestic vs external debt composition
- Fiscal policy effectiveness and impact
"""

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Advanced analytics libraries
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

class GovernmentFinanceAnalyzer:
    """Advanced Government Finance and Fiscal Policy Analysis Engine"""
    
    def __init__(self):
        self.revenue_data = {}
        self.expenditure_data = {}
        self.debt_data = {}
        self.fiscal_indicators = {}
        
    def load_fiscal_data(self, data_path="data/raw/"):
        """Load all government finance datasets"""
        
        # Revenue and expenditure data
        try:
            self.revenue_expenditure = pd.read_csv(f"{data_path}Revenue and Expenditure.csv", skiprows=2)
            print("✅ Loaded Revenue and Expenditure Data")
        except Exception as e:
            print(f"❌ Could not load Revenue and Expenditure Data: {e}")
            
        # Public debt data
        try:
            self.public_debt = pd.read_csv(f"{data_path}Public Debt.csv", skiprows=2)
            self.domestic_debt = pd.read_csv(f"{data_path}Domestic Debt by Instrument.csv", skiprows=2)
            print("✅ Loaded Public Debt Data")
        except Exception as e:
            print(f"❌ Could not load Public Debt Data: {e}")
            
        # GDP data for fiscal ratios
        try:
            self.gdp_data = pd.read_csv(f"{data_path}Annual GDP.csv", skiprows=2)
            print("✅ Loaded GDP Data")
        except Exception as e:
            print(f"❌ Could not load GDP Data: {e}")
    
    def clean_fiscal_data(self):
        """Clean and standardize fiscal data"""
        
        datasets = ['revenue_expenditure', 'public_debt', 'domestic_debt', 'gdp_data']
        
        for dataset_name in datasets:
            if hasattr(self, dataset_name):
                dataset = getattr(self, dataset_name)
                
                # Remove rows with all NaN values
                dataset = dataset.dropna(how='all')
                
                # Convert numeric columns
                numeric_cols = dataset.select_dtypes(include=[object]).columns
                for col in numeric_cols[1:]:  # Skip first column (usually time/period)
                    try:
                        # Handle common formatting issues
                        dataset[col] = dataset[col].astype(str).str.replace(',', '').str.replace('(', '-').str.replace(')', '')
                        dataset[col] = pd.to_numeric(dataset[col], errors='coerce')
                    except:
                        pass
                
                setattr(self, dataset_name, dataset)
        
        return self
    
    def analyze_revenue_trends(self):
        """Analyze government revenue trends and composition"""
        
        if not hasattr(self, 'revenue_expenditure'):
            print("Revenue and expenditure data not available")
            return None
        
        revenue_analysis = {}
        df = self.revenue_expenditure.copy()
        
        # Identify revenue columns
        revenue_cols = [col for col in df.columns if any(keyword in col.lower() 
                       for keyword in ['revenue', 'income', 'tax', 'receipt'])]
        
        if revenue_cols:
            # Calculate total revenue
            df['Total_Revenue'] = df[revenue_cols].sum(axis=1, skipna=True)
            
            # Revenue statistics
            revenue_analysis['total_revenue_stats'] = {
                'current_revenue': df['Total_Revenue'].iloc[-1] if not df['Total_Revenue'].isna().all() else None,
                'average_revenue': df['Total_Revenue'].mean(),
                'revenue_growth_rate': self._calculate_growth_rate(df['Total_Revenue']),
                'revenue_volatility': df['Total_Revenue'].std(),
                'trend': np.polyfit(range(len(df['Total_Revenue'].dropna())), df['Total_Revenue'].dropna(), 1)[0]
            }
            
            # Revenue composition analysis
            revenue_composition = {}
            for col in revenue_cols:
                if not df[col].isna().all():
                    revenue_composition[col] = {
                        'average_contribution': df[col].mean(),
                        'percentage_of_total': (df[col].sum() / df['Total_Revenue'].sum()) * 100,
                        'growth_rate': self._calculate_growth_rate(df[col]),
                        'volatility': df[col].std()
                    }
            
            revenue_analysis['revenue_composition'] = revenue_composition
        
        return revenue_analysis
    
    def analyze_expenditure_patterns(self):
        """Analyze government expenditure patterns and efficiency"""
        
        if not hasattr(self, 'revenue_expenditure'):
            print("Revenue and expenditure data not available")
            return None
        
        expenditure_analysis = {}
        df = self.revenue_expenditure.copy()
        
        # Identify expenditure columns
        expenditure_cols = [col for col in df.columns if any(keyword in col.lower() 
                           for keyword in ['expenditure', 'expense', 'spending', 'outlay'])]
        
        if expenditure_cols:
            # Calculate total expenditure
            df['Total_Expenditure'] = df[expenditure_cols].sum(axis=1, skipna=True)
            
            # Expenditure statistics
            expenditure_analysis['total_expenditure_stats'] = {
                'current_expenditure': df['Total_Expenditure'].iloc[-1] if not df['Total_Expenditure'].isna().all() else None,
                'average_expenditure': df['Total_Expenditure'].mean(),
                'expenditure_growth_rate': self._calculate_growth_rate(df['Total_Expenditure']),
                'expenditure_volatility': df['Total_Expenditure'].std(),
                'trend': np.polyfit(range(len(df['Total_Expenditure'].dropna())), df['Total_Expenditure'].dropna(), 1)[0]
            }
            
            # Expenditure composition analysis
            expenditure_composition = {}
            for col in expenditure_cols:
                if not df[col].isna().all():
                    expenditure_composition[col] = {
                        'average_spending': df[col].mean(),
                        'percentage_of_total': (df[col].sum() / df['Total_Expenditure'].sum()) * 100,
                        'growth_rate': self._calculate_growth_rate(df[col]),
                        'priority_score': self._calculate_priority_score(df[col])
                    }
            
            expenditure_analysis['expenditure_composition'] = expenditure_composition
        
        return expenditure_analysis
    
    def analyze_fiscal_balance(self):
        """Analyze fiscal balance and deficit/surplus trends"""
        
        if not hasattr(self, 'revenue_expenditure'):
            print("Revenue and expenditure data not available")
            return None
        
        fiscal_balance_analysis = {}
        df = self.revenue_expenditure.copy()
        
        # Calculate fiscal balance
        revenue_cols = [col for col in df.columns if any(keyword in col.lower() 
                       for keyword in ['revenue', 'income', 'tax', 'receipt'])]
        expenditure_cols = [col for col in df.columns if any(keyword in col.lower() 
                           for keyword in ['expenditure', 'expense', 'spending'])]
        
        if revenue_cols and expenditure_cols:
            df['Total_Revenue'] = df[revenue_cols].sum(axis=1, skipna=True)
            df['Total_Expenditure'] = df[expenditure_cols].sum(axis=1, skipna=True)
            df['Fiscal_Balance'] = df['Total_Revenue'] - df['Total_Expenditure']
            df['Fiscal_Balance_Ratio'] = df['Fiscal_Balance'] / df['Total_Revenue'] * 100
            
            # Fiscal balance statistics
            fiscal_balance_analysis['balance_stats'] = {
                'current_balance': df['Fiscal_Balance'].iloc[-1],
                'average_balance': df['Fiscal_Balance'].mean(),
                'deficit_periods': len(df[df['Fiscal_Balance'] < 0]),
                'surplus_periods': len(df[df['Fiscal_Balance'] > 0]),
                'largest_deficit': df['Fiscal_Balance'].min(),
                'largest_surplus': df['Fiscal_Balance'].max(),
                'balance_volatility': df['Fiscal_Balance'].std(),
                'trend': np.polyfit(range(len(df['Fiscal_Balance'].dropna())), df['Fiscal_Balance'].dropna(), 1)[0]
            }
            
            # Calculate fiscal ratios to GDP if GDP data available
            if hasattr(self, 'gdp_data'):
                gdp_df = self.gdp_data.copy()
                gdp_numeric = gdp_df.select_dtypes(include=[np.number]).columns
                
                if len(gdp_numeric) > 0:
                    gdp_values = gdp_df[gdp_numeric[0]]
                    min_len = min(len(df), len(gdp_values))
                    
                    if min_len > 0:
                        df_aligned = df.iloc[:min_len].copy()
                        gdp_aligned = gdp_values.iloc[:min_len]
                        
                        df_aligned['Revenue_to_GDP'] = (df_aligned['Total_Revenue'] / gdp_aligned) * 100
                        df_aligned['Expenditure_to_GDP'] = (df_aligned['Total_Expenditure'] / gdp_aligned) * 100
                        df_aligned['Balance_to_GDP'] = (df_aligned['Fiscal_Balance'] / gdp_aligned) * 100
                        
                        fiscal_balance_analysis['gdp_ratios'] = {
                            'current_revenue_to_gdp': df_aligned['Revenue_to_GDP'].iloc[-1],
                            'current_expenditure_to_gdp': df_aligned['Expenditure_to_GDP'].iloc[-1],
                            'current_balance_to_gdp': df_aligned['Balance_to_GDP'].iloc[-1],
                            'average_revenue_to_gdp': df_aligned['Revenue_to_GDP'].mean(),
                            'average_expenditure_to_gdp': df_aligned['Expenditure_to_GDP'].mean(),
                            'average_balance_to_gdp': df_aligned['Balance_to_GDP'].mean()
                        }
        
        return fiscal_balance_analysis
    
    def analyze_debt_sustainability(self):
        """Analyze public debt sustainability and composition"""
        
        debt_analysis = {}
        
        # Public debt analysis
        if hasattr(self, 'public_debt'):
            debt_df = self.public_debt.copy()
            numeric_cols = debt_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                # Calculate total debt
                debt_df['Total_Debt'] = debt_df[numeric_cols].sum(axis=1, skipna=True)
                
                debt_analysis['total_debt_stats'] = {
                    'current_debt': debt_df['Total_Debt'].iloc[-1],
                    'average_debt': debt_df['Total_Debt'].mean(),
                    'debt_growth_rate': self._calculate_growth_rate(debt_df['Total_Debt']),
                    'debt_acceleration': self._calculate_acceleration(debt_df['Total_Debt']),
                    'trend': np.polyfit(range(len(debt_df['Total_Debt'].dropna())), debt_df['Total_Debt'].dropna(), 1)[0]
                }
                
                # Debt composition
                debt_composition = {}
                for col in numeric_cols:
                    if not debt_df[col].isna().all():
                        debt_composition[col] = {
                            'current_amount': debt_df[col].iloc[-1],
                            'percentage_of_total': (debt_df[col].iloc[-1] / debt_df['Total_Debt'].iloc[-1]) * 100,
                            'growth_rate': self._calculate_growth_rate(debt_df[col])
                        }
                
                debt_analysis['debt_composition'] = debt_composition
        
        # Domestic debt analysis
        if hasattr(self, 'domestic_debt'):
            domestic_df = self.domestic_debt.copy()
            numeric_cols = domestic_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                domestic_df['Total_Domestic_Debt'] = domestic_df[numeric_cols].sum(axis=1, skipna=True)
                
                debt_analysis['domestic_debt_stats'] = {
                    'current_domestic_debt': domestic_df['Total_Domestic_Debt'].iloc[-1],
                    'domestic_debt_growth': self._calculate_growth_rate(domestic_df['Total_Domestic_Debt']),
                    'trend': np.polyfit(range(len(domestic_df['Total_Domestic_Debt'].dropna())), 
                                      domestic_df['Total_Domestic_Debt'].dropna(), 1)[0]
                }
        
        # Debt-to-GDP ratios if GDP data available
        if hasattr(self, 'gdp_data') and hasattr(self, 'public_debt'):
            gdp_df = self.gdp_data.copy()
            debt_df = self.public_debt.copy()
            
            gdp_numeric = gdp_df.select_dtypes(include=[np.number]).columns
            debt_numeric = debt_df.select_dtypes(include=[np.number]).columns
            
            if len(gdp_numeric) > 0 and len(debt_numeric) > 0:
                gdp_values = gdp_df[gdp_numeric[0]]
                total_debt = debt_df[debt_numeric].sum(axis=1, skipna=True)
                
                min_len = min(len(gdp_values), len(total_debt))
                if min_len > 0:
                    debt_to_gdp = (total_debt.iloc[:min_len] / gdp_values.iloc[:min_len]) * 100
                    
                    debt_analysis['debt_to_gdp_stats'] = {
                        'current_debt_to_gdp': debt_to_gdp.iloc[-1],
                        'average_debt_to_gdp': debt_to_gdp.mean(),
                        'debt_to_gdp_trend': np.polyfit(range(len(debt_to_gdp.dropna())), debt_to_gdp.dropna(), 1)[0],
                        'sustainability_risk': self._assess_debt_sustainability(debt_to_gdp.iloc[-1])
                    }
        
        return debt_analysis
    
    def _calculate_growth_rate(self, series):
        """Calculate compound annual growth rate"""
        if len(series.dropna()) < 2:
            return None
        
        clean_series = series.dropna()
        if clean_series.iloc[0] <= 0 or clean_series.iloc[-1] <= 0:
            return None
        
        periods = len(clean_series) - 1
        growth_rate = ((clean_series.iloc[-1] / clean_series.iloc[0]) ** (1/periods) - 1) * 100
        return growth_rate
    
    def _calculate_acceleration(self, series):
        """Calculate acceleration in growth (second derivative)"""
        if len(series.dropna()) < 3:
            return None
        
        clean_series = series.dropna()
        # Calculate second derivative
        first_diff = np.diff(clean_series)
        second_diff = np.diff(first_diff)
        return np.mean(second_diff)
    
    def _calculate_priority_score(self, expenditure_series):
        """Calculate priority score based on consistency and growth"""
        if len(expenditure_series.dropna()) < 3:
            return None
        
        growth = self._calculate_growth_rate(expenditure_series)
        if growth is None:
            return None
        
        # Higher priority = consistent positive growth
        consistency = 1 / (expenditure_series.std() + 1)  # Lower volatility = higher consistency
        priority_score = (growth * consistency) / 100
        return priority_score
    
    def _assess_debt_sustainability(self, debt_to_gdp_ratio):
        """Assess debt sustainability risk based on international thresholds"""
        if debt_to_gdp_ratio < 40:
            return "Low Risk"
        elif debt_to_gdp_ratio < 60:
            return "Moderate Risk"
        elif debt_to_gdp_ratio < 90:
            return "High Risk"
        else:
            return "Very High Risk"
    
    def create_fiscal_visualizations(self):
        """Create comprehensive fiscal policy visualizations"""
        
        visualizations = {}
        
        # 1. Revenue vs Expenditure Trends
        if hasattr(self, 'revenue_expenditure'):
            df = self.revenue_expenditure.copy()
            
            revenue_cols = [col for col in df.columns if any(keyword in col.lower() 
                           for keyword in ['revenue', 'income', 'tax'])]
            expenditure_cols = [col for col in df.columns if any(keyword in col.lower() 
                               for keyword in ['expenditure', 'expense', 'spending'])]
            
            if revenue_cols and expenditure_cols:
                df['Total_Revenue'] = df[revenue_cols].sum(axis=1, skipna=True)
                df['Total_Expenditure'] = df[expenditure_cols].sum(axis=1, skipna=True)
                df['Fiscal_Balance'] = df['Total_Revenue'] - df['Total_Expenditure']
                
                fig_fiscal = go.Figure()
                
                fig_fiscal.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(df)),
                    y=df['Total_Revenue'],
                    mode='lines+markers',
                    name='Total Revenue',
                    line=dict(color='green', width=3)
                ))
                
                fig_fiscal.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(df)),
                    y=df['Total_Expenditure'],
                    mode='lines+markers',
                    name='Total Expenditure',
                    line=dict(color='red', width=3)
                ))
                
                fig_fiscal.add_trace(go.Scatter(
                    x=fix_plotly_data(range(len(df)),
                    y=df['Fiscal_Balance'],
                    mode='lines+markers',
                    name='Fiscal Balance',
                    line=dict(color='blue', width=2)
                ))
                
                fig_fiscal.update_layout(
                    title='Government Revenue, Expenditure and Fiscal Balance',
                    xaxis_title='Period',
                    yaxis_title='Amount (Ksh Millions)',
                    template='plotly_white',
                    height=600
                )
                visualizations['fiscal_trends'] = fig_fiscal
        
        # 2. Public Debt Trends
        if hasattr(self, 'public_debt'):
            debt_df = self.public_debt.copy()
            numeric_cols = debt_df.select_dtypes(include=[np.number]).columns
            
            if len(numeric_cols) > 0:
                fig_debt = go.Figure()
                
                # Plot individual debt components
                for col in numeric_cols[:5]:  # Show top 5 components
                    if not debt_df[col].isna().all():
                        fig_debt.add_trace(go.Scatter(
                            x=fix_plotly_data(range(len(debt_df)),
                            y=debt_df[col],
                            mode='lines',
                            name=col,
                            stackgroup='one'
                        ))
                
                fig_debt.update_layout(
                    title='Public Debt Composition and Trends',
                    xaxis_title='Period',
                    yaxis_title='Debt Amount (Ksh Millions)',
                    template='plotly_white',
                    height=600
                )
                visualizations['debt_trends'] = fig_debt
        
        # 3. Debt-to-GDP Ratio
        if hasattr(self, 'gdp_data') and hasattr(self, 'public_debt'):
            gdp_df = self.gdp_data.copy()
            debt_df = self.public_debt.copy()
            
            gdp_numeric = gdp_df.select_dtypes(include=[np.number]).columns
            debt_numeric = debt_df.select_dtypes(include=[np.number]).columns
            
            if len(gdp_numeric) > 0 and len(debt_numeric) > 0:
                gdp_values = gdp_df[gdp_numeric[0]]
                total_debt = debt_df[debt_numeric].sum(axis=1, skipna=True)
                
                min_len = min(len(gdp_values), len(total_debt))
                if min_len > 0:
                    debt_to_gdp = (total_debt.iloc[:min_len] / gdp_values.iloc[:min_len]) * 100
                    
                    fig_debt_gdp = go.Figure()
                    fig_debt_gdp.add_trace(go.Scatter(
                        x=fix_plotly_data(range(len(debt_to_gdp)),
                        y=debt_to_gdp,
                        mode='lines+markers',
                        name='Debt-to-GDP Ratio',
                        line=dict(color='orange', width=3)
                    ))
                    
                    # Add sustainability thresholds
                    fig_debt_gdp.add_hline(y=60, line_dash="dash", line_color="red", 
                                          annotation_text="High Risk Threshold (60%)")
                    fig_debt_gdp.add_hline(y=40, line_dash="dash", line_color="yellow", 
                                          annotation_text="Moderate Risk Threshold (40%)")
                    
                    fig_debt_gdp.update_layout(
                        title='Public Debt-to-GDP Ratio and Sustainability Thresholds',
                        xaxis_title='Period',
                        yaxis_title='Debt-to-GDP Ratio (%)',
                        template='plotly_white',
                        height=500
                    )
                    visualizations['debt_to_gdp'] = fig_debt_gdp
        
        return visualizations
    
    def generate_fiscal_insights(self):
        """Generate key insights from fiscal analysis"""
        
        insights = []
        
        # Revenue insights
        revenue_analysis = self.analyze_revenue_trends()
        if revenue_analysis and 'total_revenue_stats' in revenue_analysis:
            rev_stats = revenue_analysis['total_revenue_stats']
            growth_rate = rev_stats.get('revenue_growth_rate')
            
            if growth_rate:
                if growth_rate > 5:
                    insights.append(f"Strong revenue growth of {growth_rate:.1f}% annually")
                elif growth_rate > 0:
                    insights.append(f"Moderate revenue growth of {growth_rate:.1f}% annually")
                else:
                    insights.append(f"Revenue declining at {abs(growth_rate):.1f}% annually")
            
            if rev_stats.get('trend', 0) > 0:
                insights.append("Government revenue is on an upward trajectory")
            else:
                insights.append("Government revenue trend is concerning - may need policy intervention")
        
        # Fiscal balance insights
        fiscal_analysis = self.analyze_fiscal_balance()
        if fiscal_analysis and 'balance_stats' in fiscal_analysis:
            balance_stats = fiscal_analysis['balance_stats']
            
            deficit_periods = balance_stats.get('deficit_periods', 0)
            surplus_periods = balance_stats.get('surplus_periods', 0)
            total_periods = deficit_periods + surplus_periods
            
            if total_periods > 0:
                deficit_ratio = deficit_periods / total_periods
                if deficit_ratio > 0.7:
                    insights.append(f"Frequent fiscal deficits ({deficit_ratio*100:.0f}% of periods) - fiscal consolidation needed")
                elif deficit_ratio > 0.5:
                    insights.append("Mixed fiscal performance with moderate deficit frequency")
                else:
                    insights.append("Generally sound fiscal management with more surpluses than deficits")
            
            # GDP ratio insights
            if 'gdp_ratios' in fiscal_analysis:
                gdp_ratios = fiscal_analysis['gdp_ratios']
                current_balance_gdp = gdp_ratios.get('current_balance_to_gdp')
                
                if current_balance_gdp:
                    if current_balance_gdp < -3:
                        insights.append(f"Large fiscal deficit of {abs(current_balance_gdp):.1f}% of GDP - requires attention")
                    elif current_balance_gdp < 0:
                        insights.append(f"Manageable fiscal deficit of {abs(current_balance_gdp):.1f}% of GDP")
                    else:
                        insights.append(f"Fiscal surplus of {current_balance_gdp:.1f}% of GDP - strong fiscal position")
        
        # Debt sustainability insights
        debt_analysis = self.analyze_debt_sustainability()
        if debt_analysis and 'debt_to_gdp_stats' in debt_analysis:
            debt_stats = debt_analysis['debt_to_gdp_stats']
            current_debt_gdp = debt_stats.get('current_debt_to_gdp')
            sustainability_risk = debt_stats.get('sustainability_risk')
            
            if current_debt_gdp:
                insights.append(f"Current debt-to-GDP ratio: {current_debt_gdp:.1f}% - {sustainability_risk} sustainability")
            
            debt_trend = debt_stats.get('debt_to_gdp_trend', 0)
            if debt_trend > 1:
                insights.append("Debt-to-GDP ratio is rising rapidly - debt management strategy needed")
            elif debt_trend > 0:
                insights.append("Debt-to-GDP ratio is increasing moderately")
            else:
                insights.append("Debt-to-GDP ratio is stable or declining - positive trend")
        
        return insights

# Initialize analyzer
analyzer = GovernmentFinanceAnalyzer()

print("🏛️ Government Finance and Fiscal Policy Analysis Notebook")
print("=" * 60)
print("Comprehensive analysis of Kenya's government finances, fiscal policy,")
print("and public debt sustainability.")
print()

# Load and analyze data
print("📊 Loading fiscal data...")
analyzer.load_fiscal_data()
analyzer.clean_fiscal_data()

print("\n💰 Analyzing revenue trends...")
revenue_analysis = analyzer.analyze_revenue_trends()

print("\n💸 Analyzing expenditure patterns...")
expenditure_analysis = analyzer.analyze_expenditure_patterns()

print("\n⚖️ Analyzing fiscal balance...")
fiscal_balance_analysis = analyzer.analyze_fiscal_balance()

print("\n📈 Analyzing debt sustainability...")
debt_analysis = analyzer.analyze_debt_sustainability()

print("\n🔍 Generating fiscal policy insights...")
insights = analyzer.generate_fiscal_insights()
for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

print("\n📊 Creating visualizations...")
visualizations = analyzer.create_fiscal_visualizations()
print(f"Generated {len(visualizations)} visualizations")

print("\n✅ Government Finance and Fiscal Policy Analysis Complete!")
print("Key outputs:")
print("- Revenue trend analysis and composition")
print("- Expenditure efficiency assessment")
print("- Fiscal balance and deficit analysis")
print("- Public debt sustainability metrics")
print("- Debt-to-GDP ratio tracking")
print("- Fiscal policy effectiveness indicators")
print("- Interactive fiscal visualizations")